In [1]:
import json
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch
import pandas as pd

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/home/codespace/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
with open('../data/vietnamese_rag/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
df_ground_truth = pd.read_csv('../data/vietnamese_rag/ground_truth_data/ground_truth_data.csv')

ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': "Minh Tú đã gặp khó khăn gì trong thử thách đi catwalk tại Asia's Next Top Model mùa 5?",
 'Group': 'General',
 'document': '75fafd29'}

In [10]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "group": {"type": "keyword"},
            "context": {"type": "text"},
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "id": {"type": "keyword"},
            "context_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_context_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "vietnamese-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vietnamese-questions'})

In [11]:
def load_vectors(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def process_documents(documents, index_name, es_client):
    document_question_vector_list = load_vectors(f'../data/vietnamese_rag/question_vector_pickle/question_vector.pkl')
    document_answer_vector_list = load_vectors(f'../data/vietnamese_rag/answer_vector_pickle/answer_vector.pkl')
    document_context_vector_list = load_vectors(f'../data/vietnamese_rag/context_vector_pickle/context_vector.pkl')
    document_qta_vector_list = load_vectors(f'../data/vietnamese_rag/question_context_answer_vector_pickle/question_context_answer_vector.pkl')
    for j in range(len(documents)):
        documents[j]['question_vector'] = document_question_vector_list[j]['question_vector']
        documents[j]['answer_vector'] = document_answer_vector_list[j]['answer_vector']
        documents[j]['context_vector'] = document_context_vector_list[j]['context_vector']
        documents[j]['question_context_answer_vector'] = document_qta_vector_list[j]['question_context_answer_vector']
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)
process_documents(documents, index_name, es_client)

100%|█████████| 6089/6089 [02:19<00:00, 43.74it/s]


In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [13]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, group, k=60):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "group": group
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["context", "question", "answer"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "group": group
                }
            }
        }
    }

    knn_results = es_client.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [14]:
def context_vector_hybryd(q):
    question = q['question']
    group = q['Group']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('context_vector', question, v_q, group)

In [15]:
def question_context_answer_vector_hybryd(q):
    question = q['question']
    group = q['Group']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('question_context_answer_vector', question, v_q, group)

In [16]:
def answer_vector_hybryd(q):
    question = q['question']
    group = q['Group']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('answer_vector', question, v_q, group)

In [17]:
def question_vector_hybryd(q):
    # print(q)
    question = q['question']
    group = q['Group']
    if not isinstance(question, str):
        print(question)
        raise TypeError(f"Expected question to be a string, but got {type(question)}")
    v_q = model.encode(question)
    return elastic_search_hybrid_rrf('question_vector', question, v_q, group)

In [20]:
evaluate(ground_truth, question_vector_hybryd)

100%|███████| 29203/29203 [26:54<00:00, 18.09it/s]


{'hit_rate': 0.9536349005239187, 'mrr': 0.8312433654076612}